# GLOBALS AND INITIAL PARAMETERS

In [86]:
from emot import emo_unicode
from keras.callbacks import Callback, EarlyStopping, CSVLogger, ReduceLROnPlateau, ModelCheckpoint
from keras.layers import Dense, Activation, BatchNormalization, Embedding, Conv1D, Dropout, LSTM, TimeDistributed, GRU, Masking 
from keras.models import load_model, Sequential
from keras.optimizers import Adam
from keras.utils import to_categorical
from nltk.tokenize import word_tokenize #, wordpunct_tokenize
from sklearn.model_selection import train_test_split
from time import sleep
import csv
import matplotlib
import numpy as np
import re
import sys
import string

matplotlib.use('Agg')
from matplotlib import pyplot as plt 

# CHANGE PATH TO MODEL OUTPUTS HERE BASED ON YOUR LOCAL CONFIG
BASE_PATH = '/Users/goldenzenith/Dropbox (MIT)/6.867_saved_models/'
FOLDER = 'GRU_WORDWORD/'
PATH = BASE_PATH + FOLDER
print 'Model files will be saved to:', PATH

# plots, logs, weight files will be based on this
# plz name descriptively, e.g. lstm_512_charchar
MODEL_NAME = 'gru_512_wordword' 

# approach
CHAR_BY_CHAR = False # True for character-by-character training
CUSTOM_NAMES = True # True to use CUSTOM_NAME token (not '') for tagged FB names

if CUSTOM_NAMES:
    COMMENTS_FILEPATH = "csv_data/CUSTOM_NAMES/custom_name_token_comments.csv"
else:
    COMMENTS_FILEPATH = "csv_data/FILTERED_NAMES/filtered_names_comments.csv"

# paths with data based on filtering done in adapted version of Facebook post scraping
STATUS_FILEPATH = "csv_data/beaverconfessions_facebook_statuses.csv" 

# parameters/settings
# MAX_LEN is defined later
EMBEDDING = False # True to use CNN
if EMBEDDING:
    CONV_NB_FILTERS = 100
    CONV_KERNEL_SIZE = 3
LSTM_MODEL = False # False to use GRU
HIDDEN_UNITS = 512
STEP_SIZE = 1
WINDOW_SIZE = 10 

# sample weights
WEIGHT_BY_REACTIONS = False # False to give all samples same weight
LIKES_ONLY = False # True for only likes, not all reactions

# special symbols
CUSTOM_NAME = '<name>'
CUSTOM_NUMBER = '<number>'
PADDING_SYMBOL = '<padding>'
STOP_SYMBOL = '<stahp>'
CUSTOM_SYMBOLS = [CUSTOM_NAME, CUSTOM_NUMBER, PADDING_SYMBOL, STOP_SYMBOL]

# PRE-PROCESSING (TOKENIZATION) 

In [74]:
escaped_punctuation = re.escape(string.punctuation)
# print 'Escaped punctuation:', escaped_punctuation

# insert OR pipe before each punctuation mark
xor_punctuation = '|'.join('{}{}+'.format(escaped_punctuation[x], escaped_punctuation[x+1]) for x in range(0, len(escaped_punctuation), 2))
# print 'Delimited punctuation:', xor_punctuation

# build regex with variable (order matters!)
nm = CUSTOM_NAME.lower()
nb = CUSTOM_NUMBER.lower()
pd = PADDING_SYMBOL.lower()
sp = STOP_SYMBOL.lower()
course_number = '\d+\.\d{2,3}'
multiple_numbers = '\d+'
emoticon_pattern = '|'.join(emoticon for emoticon in emo_unicode.EMOTICONS)
space = '\s'
regex_expr = r'(' + '|'.join([nm, nb, pd, sp, course_number, multiple_numbers, emoticon_pattern, xor_punctuation, space]) + r')'
# print '\nRegex expression:', regex_expr

def replace_digit_with_token(string): 
    return CUSTOM_NUMBER.lower() if string.isdigit() else string

def tokenize_str(string, replace_digits=True):
    """
    NLTK tokenizers (word_tokenize, wordpunct_tokenize) are insufficient, 
    as emoticons and course #s, e.g. 6.111, are important to our dataset.
    
    This is a custom tokenizer to retain such items in the vocab,
        but split up other words containing numbers/punctuation, e.g. 3pm.
        
    Note that these texts are lowercased by default prior to tokenization.
    """
    if CHAR_BY_CHAR:
        tokens = list(string)
        
    else:
        tokens = re.split(regex_expr, string.lower())

        # filter out spaces
        tokens = [token for token in tokens if token not in ["", " "]]

        # tokens with *just* digits are mapped to <NUMBER> by default
        if replace_digits:
            tokens = map(lambda x: replace_digit_with_token(x), tokens)

    return tokens

# attempt to catch major cases in our dataset
sentence = CUSTOM_NAME + '6.867test. @7:30pm in 54-100. The profs, they\'re coming in 3..2..1. LETS DOOO THIS!!! >;)' + STOP_SYMBOL + PADDING_SYMBOL
print 'Test case:', tokenize_str(sentence)

In [75]:
def load_dataset_and_vocabulary(filepath, vocabulary, datatype, max_nb_tokens, reactions):
    """
    Read individual sentences into memory, and generate vocabulary.
    
    If CHAR_BY_CHAR, the vocabulary will hold single characters, 
        e.g. a-zA-Z and punctuation.
    Else, it will contain whole words and Unicode 'phrases',
        e.g. :\'(, as split by our custom tokenizer.
    """
    dt = "{}_message".format(datatype)
    reaction_header = 'num_likes' if LIKES_ONLY else 'num_reactions'    
    
    with open(filepath, "rU") as csvfile:
        reader = csv.DictReader(csvfile)
        tokens = []
        
        for status in reader:
            # we distinguish between comments and statuses with header cols in the CSVs
            
            if dt not in status:
                # 2 rows are read at once from STATUS_FILEPATH for some reason...
                # handled with monkey patching 
                msg1, msg2 = status.items()[1]
                
                # append stop symbol so model has explicit marker for ending
                msg1 += STOP_SYMBOL
                msg2 += STOP_SYMBOL
                
                # we define elts as the unit we are training on, e.g. char vs. word 
                elts_in_sentence1 = tokenize_str(msg1)
                elts_in_sentence2 = tokenize_str(msg2)
                
                max_nb_tokens = max(max_nb_tokens, max(len(elts_in_sentence1), len(elts_in_sentence2)))
                 
                for elt in elts_in_sentence1:
                    vocabulary.add(elt)
                for elt in elts_in_sentence2:
                    vocabulary.add(elt) 
                    
                # build list of tokens in local memory
                tokens.append(elts_in_sentence1)
                tokens.append(elts_in_sentence2)
            else:
                msg = status[dt]
                nb_reactions = float(status[reaction_header])
                
                msg += STOP_SYMBOL 
                elts_in_sentence = tokenize_str(msg)
                max_nb_tokens = max(max_nb_tokens, len(elts_in_sentence))
                    
                for elt in elts_in_sentence:
                    vocabulary.add(elt)
                    
                tokens.append(elts_in_sentence)
                reactions.append(nb_reactions)
                    
    return vocabulary, tokens, max_nb_tokens, reactions
    
# load comments and statuses separately at first
max_nb_tokens = 0

vocabulary, comment_tokens, max_nb_tokens, comment_reactions = \
    load_dataset_and_vocabulary(COMMENTS_FILEPATH, set([]), "comment", max_nb_tokens, [])

vocabulary, status_tokens, max_nb_tokens, reactions = \
    load_dataset_and_vocabulary(STATUS_FILEPATH, vocabulary, "status", max_nb_tokens, comment_reactions)

MAX_LEN = max_nb_tokens # includes stop symbol

max_nb_reactions = max(reactions)
print '\nMax # of reactions:', max_nb_reactions

print 'Max # of tokens in a sentence:', MAX_LEN
print '\nSample of 10 processed sentences:'
for status in status_tokens[:5]:
    print status
    
# create vocabulary of characters found in data
if not CHAR_BY_CHAR:
    vocabulary.add(CUSTOM_NAME)
    vocabulary.add(CUSTOM_NUMBER)
    
# these symbols did not exist in the original training set,
# so we can safely add them 
# NOTE: changed from insert(0,_), since we were adding duplicates
vocabulary.add(STOP_SYMBOL)
vocabulary.add(PADDING_SYMBOL)
vocabulary = sorted(list(vocabulary))

NB_CLASSES = len(vocabulary)

tokens_indices = dict((t, i) for i, t in enumerate(vocabulary))
indices_tokens = {v: k for k, v in tokens_indices.iteritems()}
PADDING_INDEX = tokens_indices[PADDING_SYMBOL]
STOP_INDEX = tokens_indices[STOP_SYMBOL]

print '\nPadding index:', PADDING_INDEX, 'and stop index:', STOP_INDEX

input_type = "character by character" if CHAR_BY_CHAR else "word by word"
# print "For", input_type, ", we have this vocabulary:", vocabulary, "\n of size", NB_CLASSES

# BUILD TRAINING SETS

In [76]:
# merge comments and statuses
tokens = []
tokens.extend(comment_tokens)
tokens.extend(status_tokens)
tokens = np.array(tokens)

print 'Sample tokenized sentence:'
print tokens[0]
print

def generate_X_and_Y(sentences):
    """
    X = sub_sentences, Y = next_sub_sentences
    Y is simply X shifted over by step_size.
    We want sub sentences per sentence to create multiple samples.
    """
    sub_sentences = []
    next_sub_sentences = []
    
    sub_reactions = []
    
    for sentence_nb, sentence in enumerate(sentences):
        nb_sentence_reactions = reactions[sentence_nb]
        
        for i in range(0, len(sentence) - WINDOW_SIZE, STEP_SIZE):
            sub_reactions.append(nb_sentence_reactions)
            sub_sentences.append(sentence[i : i+WINDOW_SIZE])
            next_sub_sentences.append(sentence[(i+STEP_SIZE) : (i+STEP_SIZE)+WINDOW_SIZE])
    
    return sub_sentences, next_sub_sentences, sub_reactions

X, Y, Reacts = generate_X_and_Y(tokens)
nb_samples = len(X)
print 'Sample X:', X[0]
print 'Sample Y:', Y[0]
print 

def convert_tokens_to_int(X, Y, nb_samples):
    """
    Convert token to integer representations.
    
    Populate label matrices with ints,
    so result is padded and ready for training.
    """
    # IGNORE THIS COMMENT but DON'T REMOVE -> extra 1 represents space for the stop symbol
    input_shape = (nb_samples, MAX_LEN)
    X_labels = np.zeros(input_shape)
    y_labels = np.zeros(input_shape)

    for sample_nb in range(nb_samples):
        x_label = map(lambda x: tokens_indices[x], X[sample_nb]) 
        y_label = map(lambda x: tokens_indices[x], Y[sample_nb])

        X_labels[sample_nb][:len(x_label)] = x_label
        X_labels[sample_nb][len(x_label):] = PADDING_INDEX
        y_labels[sample_nb][:len(y_label)] = y_label
        y_labels[sample_nb][len(y_label):] = PADDING_INDEX
    
    return X_labels, y_labels

X_labels, y_labels = convert_tokens_to_int(X, Y, nb_samples)

print 'Sample X labels:\n', X_labels[:1]
print 'Sample Y labels:\n', y_labels[:1]
print '\nTOTAL NB OF TRAINING SAMPLES:', nb_samples

# split data into train and val sets
X_train_labels, X_val_labels, y_train_labels, y_val_labels = \
    train_test_split(X_labels, y_labels, test_size=0.3)
X_train_reacts, X_val_reacts = \
    Reacts[:len(X_train_labels)], Reacts[len(X_train_labels):]


# BATCH GENERATOR

In [77]:
def validate_data(X, y):
    # assert at least 1 nonzero value in each OHE sample
    assert(False not in np.any(X>0, axis=1))
    assert(False not in np.any(y>0, axis=1))

def sampling_generator(epoch_size, batch_size, validation=False):  
    """
    Takes in labels of int values, e.g. [1. 2. 5. 9. 1. 0. 0. ...]
    These labels are already padded with 0s to a predetermined maxlen.
    We convert each sample X and y to one hot versions, and create batches.
    Batch generation is necessary to avoid MemoryError.
    """
    if validation:
        X_labels = X_val_labels
        y_labels = y_val_labels 
        react_labels = X_val_reacts
    else:
        X_labels = X_train_labels
        y_labels = y_train_labels
        react_labels = X_train_reacts
    
    while True:
        # hand off data in batches
        for i in range(int(epoch_size/batch_size)):
            start = i * batch_size
            end = min(start + batch_size, epoch_size)
            true_batch_size = end - start

            # fresh batch
            batch_X = []
            batch_Y = []
            batch_shape = (true_batch_size, MAX_LEN, NB_CLASSES)
            
            X_label_sample = X_labels[start:end]
            X_label_sample_reacts = react_labels[start:end]

            # one hot encode
            if EMBEDDING:
                batch_X = X_label_sample
            else:
                batch_X = to_categorical(X_label_sample, num_classes=NB_CLASSES).reshape(batch_shape)
            
            batch_Y = to_categorical(y_labels[start:end], num_classes=NB_CLASSES).reshape(batch_shape)                    
            
            total_nb_one_hots = X_label_sample.size
            sample_weights = np.ones((total_nb_one_hots, 1))
            
            temp_reshaped_Xs = np.ravel(X_label_sample).reshape(-1)
            padded_positions = np.where(temp_reshaped_Xs == PADDING_INDEX)
            sample_weights[padded_positions] = 0 # 28100 x 1
            sample_weights = sample_weights.reshape(true_batch_size, MAX_LEN)
            
            if WEIGHT_BY_REACTIONS:
                # turn into col vector
                react_weights = np.array(X_label_sample_reacts).reshape(-1,1)
                react_weights = (react_weights / max_nb_reactions) + 1
                
                sample_weights = np.multiply(sample_weights, react_weights)
            
            yield (batch_X, batch_Y, sample_weights)
            
# next(sampling_generator(200, 128, validation=False))

# MODEL ARCHITECTURE

In [78]:
def build_model():
    print('Building model...')
    model = Sequential()
    
    if EMBEDDING:
        model.add(Embedding(input_dim=NB_CLASSES, output_dim=64, input_length=MAX_LEN))
        model.add(Conv1D(CONV_NB_FILTERS, CONV_KERNEL_SIZE, padding='same', activation='linear', strides=1))
        model.add(Activation('relu'))
        next_inp_shape = (MAX_LEN, CONV_NB_FILTERS)
    else:
        next_inp_shape = (MAX_LEN, NB_CLASSES)        
    
    if LSTM_MODEL:
        model.add(LSTM(512, return_sequences=True, input_shape=next_inp_shape))
        model.add(Dropout(0.2))
        model.add(LSTM(512, return_sequences=True))
        model.add(TimeDistributed(Dense(NB_CLASSES)))
    else:
        model.add(GRU(512, return_sequences=True, input_shape=next_inp_shape))
        model.add(Dropout(0.2))
        model.add(GRU(512, return_sequences=True))
        model.add(TimeDistributed(Dense(NB_CLASSES)))

    model.add(Activation('softmax'))
    
    # to apply sample weights to metrics, specify weighted_metrics=[list of metrics]
    model.compile(loss='categorical_crossentropy', optimizer=Adam(clipnorm=1.0), sample_weight_mode="temporal") 
    model.summary()
    return model

model = build_model()

# CALLBACKS (bells + whistles)

In [87]:
class PlotHistory(Callback):
    def __init__(self, path, run_name):
        self.path = path
        self.run_name = run_name 

    def on_train_begin(self, logs=None):
        self.epoch = []
        self.history = {}

    def on_epoch_end(self, epoch, logs=None):
        logs = logs or {}
        self.epoch.append(epoch)

        for k, v in logs.items():
            self.history.setdefault(k, []).append(v)

        # create loss and perplexity plot
        loss_handles = []
        train_perp = []
        val_perp = []
        for key in self.history:           
            if key != "lr":
                l, = plt.plot(self.history[key], label=key)
                loss_handles.append(l)

        plt.title('Losses and metrics for {}'.format(self.run_name))    
        plt.ylabel('loss')
        plt.yscale('symlog')
        plt.legend(["Train Loss","Val Loss"], fontsize=8, loc='upper right')          
        plt.savefig('{}_plot.jpg'.format(self.path+self.run_name))        
        plt.clf()
    
checkpointer = ModelCheckpoint(filepath='{}.hdf5'.format(PATH+MODEL_NAME), verbose=1, save_best_only=True)
csv_logger = CSVLogger('{}.log'.format(PATH+MODEL_NAME))
early_stopping = EarlyStopping(monitor='val_loss', patience=5, min_delta=0.001, verbose=1)
plot_history = PlotHistory(PATH, MODEL_NAME)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, verbose=1, epsilon=1e-4, min_lr=1E-6) 

# PERPLEXITY

In [92]:
def plotPerplexity(path, name):
    f = open(path+name+'.log','r')
    epoch_X = []
    train_perplex = []
    val_perplex = []

    for line in f.readlines()[1:]:
        data = line.strip().split(",")
        epoch_X.append(int(data[0]))
        train_perplex.append(2**(float(data[1])))
        val_perplex.append(2**(float(data[3])))

    fig = plt.figure()
    ax = plt.axes()

    ax.plot(epoch_X, train_perplex)
    ax.plot(epoch_X, val_perplex)
    plt.legend(['Train perplexity', 'Val perplexity'], fontsize=8, loc='upper right')
    plt.title('Perplexities for {}'.format(name))
    plt.savefig('{}_perplexity.jpg'.format(path+name))
    plt.clf()

# TRAIN MODEL

In [95]:
# True for print statements to show in terminal instead
PRINT_TRAIN_PROGRESS_TO_TERMINAL = True 
DEBUG_MODE = False

if PRINT_TRAIN_PROGRESS_TO_TERMINAL:
    reload(sys)    

if DEBUG_MODE:
    small_samples = 200
    nb_train_samples = small_samples
    nb_val_samples = small_samples
    batch_size = 128
    NB_EPOCHS = 2
else:
    nb_train_samples = len(X_train_labels)
    nb_val_samples = len(X_val_labels)
    NB_EPOCHS = 20

print 'Total training samples:', nb_train_samples
print 'Total val samples:', nb_val_samples

In [90]:
def train(model):
    # shuffling of training data True by default
    print 'Training...'
    history = model.fit_generator(
        sampling_generator(nb_train_samples, batch_size), 
        steps_per_epoch=nb_train_samples/batch_size,
        epochs=NB_EPOCHS,
        verbose=1,
        validation_data=sampling_generator(nb_val_samples, batch_size, validation=True),
        validation_steps=nb_val_samples/batch_size,
        callbacks=[early_stopping, reduce_lr, csv_logger, checkpointer, plot_history])

    hist = history.history
    plotPerplexity(PATH, MODEL_NAME)
#     print 'Loss:', hist['loss'][0], 'and val loss:', hist['val_loss'][0]
    print '\nFull training history:\n', hist

In [93]:
train(model)

# LOAD MODEL

In [ ]:
model_path = '/Users/goldenzenith/Dropbox\ \(MIT\)/6.867_saved_models/GRU_wordword/gru_512_wordword.hdf5'
model = load_model(model_path)
print model


# TEXT GENERATION

In [ ]:
def convert_sentence_to_ohe(sentence):
    x_label = map(lambda x: tokens_indices[x], sentence)
    confession = np.zeros(MAX_LEN+1)
    confession[:len(x_label)] = x_label
    ohe_x = to_categorical(confession, num_classes=NB_CLASSES)
    return np.expand_dims(ohe_x, axis=0)

from scipy.misc import logsumexp

def log_softmax(vec):
    return vec - logsumexp(vec)

def softmax(vec):
    return np.exp(log_softmax(vec))

def generate_confession(model, seed_string, seed_list, temperature, sample, thresh, confidence):
    # nb chars to preserve
    if CHAR_BY_CHAR:
        orig_len = len(seed_string) 
    else:
        orig_len = len(seed_list) 
        
    window_str = seed_string
    window_list = seed_list
    final_str = seed_string 
        
    for unit_nb in range(orig_len, MAX_LEN):
        if CHAR_BY_CHAR:
            x = convert_sentence_to_ohe(window_str)
        else:
#             print "list here is", window_list
            x = convert_sentence_to_ohe(window_list)
    
        if sample:
            # get next char
            next_char_idx = min(WINDOW_SIZE, unit_nb) - 1
        
            # helper function to sample an index from a probability array
            # indexing into 3D matrix -- get row of probs for single char at next_char_idx
            preds = model.predict(x)[0, next_char_idx, :]
            preds = np.asarray(preds).astype('float64')
            preds = np.log(preds) / temperature
            exp_preds = np.exp(preds)
            preds = exp_preds / np.sum(exp_preds)
            probas = np.random.multinomial(1, preds, 1)
            next_token = np.argmax(probas)
        
            maxConf = np.max(preds)
#             print "max conf",maxConf
            if maxConf > thresh:
                confidence += 1
        else:
            print x.shape
            preds = np.squeeze(model.predict(x),axis=0) # otherwise wrapped in (1,maxlen+1,len(chars))
            best_tokens = np.argmax(preds, axis=1)
#             y = preds[0,:]
#             plt.plot(y.tolist())
#             plt.savefig("testttt.jpg")
#             plt.clf()

            if unit_nb >= WINDOW_SIZE:
                maxConf = np.max(preds[WINDOW_SIZE-1])
            else:
                maxConf = np.max(preds[unit_nb-1])
                
            if maxConf > thresh:
                confidence += 1
            
            # unit_nb-1 because we want prev val in y matrix (best_tokens). In training we treat y as a shifted 
            # version of x hence offset -1 here
            if unit_nb >= WINDOW_SIZE:
                next_token = best_tokens[WINDOW_SIZE-1]
            else:
                next_token = best_tokens[unit_nb-1]
                
        # stop symbol
        if next_token == STOP_INDEX:
            if CHAR_BY_CHAR:
                confession_list = final_str
            else:
                confession_list = final_str.split()
            print "confidence here",confidence 
            print "len of confession here",len(confession_list)
            return final_str,confidence/float(len(confession_list))
        
        next_char = indices_tokens[next_token]
        
        if CHAR_BY_CHAR:
            if len(window_str) == WINDOW_SIZE:
                moveConf = window_str[1:] + next_char
                window_str = moveConf
            else:
                window_str += next_char
            final_str += next_char
        else:
            window_list.append(next_char)
            if len(window_list)-1 == WINDOW_SIZE:
                moveConf = window_list[1:]
                window_list = moveConf

            if window_str != '#':
                window_str += ' '+next_char
                final_str += ' '+next_char
            else:
                window_str += next_char
                final_str += next_char
            
        print "final str:",final_str
        
#     print "\n this",final_str,confidence
                
    if CHAR_BY_CHAR:
        confession_list = final_str
    else:
        confession_list = final_str.split()
    print "confidence there",confidence
    print "len of confession there",len(confession_list)
    return final_str,confidence/float(len(confession_list))


# Run and place into .txt file
ARG_MAX = True
if ARG_MAX:
    filename = MODEL_NAME+'_preds.txt'
    sample_bool = False
else:
    filename = MODEL_NAME+'_temp_0_5_preds.txt'
    sample_bool = True
    
# seed_strings_cc=["#", "#1", "#12", "#124", "#1246", "#12465", "#9999", 
#              "#1 Hi", "#12 Hi", "#124 Hi", "#1246 Hi", "#1293 Hi m",
#              "#2568 ML", "#3476 Hixz", "#1321 I'm", "#2346 :)", "#5876 7:30am",
#              "#9235 6.01", "#1246 6.867"]
seed_strings_cc=["#", "#3476", "#3476 i",
             "#3476 i'm", "#3476 :)", 
             "#3476 7:30am",
             "#3476 6.01"]
seed_strings_ww=["#", "# CUSTOM_NUMBER", "# CUSTOM_NUMBER i",
             "# CUSTOM_NUMBER i ' m", "# CUSTOM_NUMBER :)", 
             "# CUSTOM_NUMBER CUSTOM_NUMBER :3 CUSTOM_NUMBER am",
             "# CUSTOM_NUMBER 6.01"]
conf_list = []
confidence = 0


with open(filename, 'w') as f:
    print "Writing to",filename
    if CHAR_BY_CHAR:
        seed_list = seed_strings_cc
    else:
        seed_list = seed_strings_ww
        
    for seed_string in seed_list:
        print "seed_string:",seed_string
        seed_list = seed_string.split()
        #set to true for temperature sampling version
        confession, confidence = generate_confession(model, seed_string, seed_list, 0.5, sample_bool, 0.8, 0) 
        print "confession:",confession
        print "confidence:", confidence
        conf_list.append(confidence)
        
#         if len(conf_list) >1:
#             break
        f.write(confession)
        f.write("\n")
        
    avg_conf = sum(conf_list)/len(conf_list)
    f.write("confidence list: "+ str(conf_list))
    f.write("\n")
    f.write("average confidence: "+ str(avg_conf))
        
print "confidence for test set is:",conf_list